### Script to calculate the diva3D products

* The script loops over the variable names in _varname_
* the signal-to-noise (e) and correlation length (l) parameters calculated in the previous script are loaded from
    * /_varname_/*varname*_newe.txt
    * /_varname_/*varname*_newl.txt
* these parameters are first filtered (only l < 500 000 and e < 15 are retained) and then averaged
* with these new averaged parameters the diva product is calculated: 1 diva interpolated map
    * per species
    * per season
    * per year
* the output netcdf files are stored in /_varname_/netcdf_all/

In [1]:
BLAS.set_num_threads(1)
versioninfo()
print(join(["$p: $v\n" for (p,v) in Pkg.installed()]))

Julia Version 0.6.4
Commit 9d11f62bcb (2018-07-09 19:09 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Xeon(R) CPU E5-2630L 0 @ 2.00GHz
  WORD_SIZE: 64
  BLAS: libopenblas (USE64BITINT DYNAMIC_ARCH NO_AFFINITY Sandybridge MAX_THREADS=16)
  LAPACK: libopenblas64_
  LIBM: libopenlibm
  LLVM: libLLVM-3.9.1 (ORCJIT, sandybridge)
Libz: 0.4.0
AxisAlgorithms: 0.3.0
OffsetArrays: 0.6.0
HTTP: 0.6.12
CondaBinDeps: 0.1.0
OceanPlot: 0.0.0-
Nullables: 0.0.6
ZMQ: 0.6.3
DataStructures: 0.8.4
Compat: 1.0.0
EzXML: 0.7.1
ShowItLikeYouBuildIt: 0.2.0
SpecialFunctions: 0.6.0
Blosc: 0.5.0
ZipFile: 0.6.0
FixedPointNumbers: 0.4.6
SHA: 0.5.7
Missings: 0.2.10
MAT: 0.4.0
HDF5: 0.9.3
ColorTypes: 0.6.7
BufferedStreams: 0.4.1
MbedTLS: 0.5.12
SortingAlgorithms: 0.2.1
Conda: 1.0.0
PyCall: 1.17.1
WoodburyMatrices: 0.3.0
JSON: 0.17.2
StatsBase: 0.23.1
IJulia: 1.9.1
PyPlot: 2.6.0
BinDeps: 0.8.8
Parameters: 0.9.1
DIVAnd: 2.0.0+
Mustache: 0.3.3
Reexport: 0.1.0
CMakeWrapper: 0.1.0
URIParser: 0.3.1
In

In [2]:
# change working directory
@show pwd()
cd("//data/20180306_OOPS/")
@show pwd()

"/data/20180306_OOPS"

In [3]:
using DIVAnd
using PyPlot
using NCDatasets
using DataStructures

# Load a more efficient version of sparse matrix multiplication
include(joinpath(Pkg.dir("DIVAnd"),"src","override_ssmult.jl"))

pwd() = "/home/lennerts/Diva_product_scripts_2018"
pwd() = "/data/20180306_OOPS"


In [4]:
function yearlists_(dataset_range, total_window_yrs)
    # dataset_range = 2000:2012
    # total_window_yrs = 10
    # will return: [2000:2009, 2001:2010, 2002:2011, 2003:2012]
    
    n_windows = length(dataset_range) - total_window_yrs + 1

    a = Array{UnitRange{Int64}}(n_windows)

    for i = 1:n_windows
        a[i] =  dataset_range[i]:(dataset_range[i] + total_window_yrs -1)
    end
    return(a)
end

yearlists_ (generic function with 1 method)

In [5]:
function plotres(timeindex,sel,fit,erri) # fit = interpolated field
    tmp = copy(fit)
    tmp[erri .> .3] = NaN; # only plotting where error < 0.3
    figure(figsize = (10,8))
    subplot(2,1,1)
    title("$(timeindex) - surface")
    
    # select the data near the surface
    selsurface = sel .& (depth .< 5)
    vmin = minimum(value[selsurface])
    vmax = maximum(value[selsurface])
    
    # plot the data
    scatter(lon[selsurface],lat[selsurface],10,value[selsurface];
            vmin = vmin, vmax = vmax)
    xlim(minimum(lonr),maximum(lonr))
    ylim(minimum(latr),maximum(latr))
    colorbar()
    contourf(bx,by,b', levels = [-1e5,0],colors = [[.5,.5,.5]])
    
    # plot the analysis
    subplot(2,1,2)
    pcolor(lonr,latr,tmp[:,:,1]';
           vmin = vmin, vmax = vmax)
    colorbar()
    contourf(bx,by,b', levels = [-1e5,0],colors = [[.5,.5,.5]])
end

plotres (generic function with 1 method)

In [6]:
bathname = "gebco_30sec_4.nc"

if !isfile(bathname)
    download("https://b2drop.eudat.eu/s/ACcxUEZZi6a4ziR/download",bathname)
else
    info("Bathymetry file already downloaded")
end

bathisglobal = true

true

INFO: Bathymetry file already downloaded


In [7]:
dx = dy = 0.1
lonr = -75.:dx:20.; # the range of longitudes (start:step:end)
latr = 35.:dy:75.; # the range of latitudes (start:step:end)
depthr = [-1., 1.] # put . always! (otherwise integer and error in DIVA)

# create mask
# mask,(pm,pn),(xi,yi) = divand_rectdom(lonr,latr)
# I changed the line below after error with diva3d
mask,(pm,pn),(xi,yi) = DIVAnd.domain(bathname, true, lonr, latr)
bx,by,b = load_bath(bathname,true,lonr,latr)


(-75.0:0.1:20.0, 35.0:0.1:75.0, [2475.89 1803.55 … 290.672 483.172; 2698.02 2466.61 … 291.781 466.875; … ; 3013.61 2988.84 … 173.094 126.891; 2998.63 2965.13 … 146.625 140.734])

In [8]:
# Months and years
monthlists = [
    [1,2,3],
    [4,5,6],
    [7,8,9],
    [10,11,12]
]
yearlists = yearlists_(1958:2016, 1);

In [9]:
ncglobalattrib = OrderedDict(
    # Name of the project (SeaDataCloud, SeaDataNet, EMODNET-Chemistry, ...)
    "project" => "EMODnet Biology",

    # URN code for the institution EDMO registry,
    # e.g. SDN:EDMO::422 -> Flanders Marine Institute (VLIZ)
    "institution_urn" => "SDN::EDMO::422",

    # Production group
    "production" => "Flanders Marine Institute",

    # Name and emails from authors
    "Author_e-mail" => "Lennert Schepers <lennert.schepers@vliz.be>",

    # Source of the observation
    "source" => "SAHFOS / Marine Biological Association (UK)",

    # Additional comment
    "comment" => "...",

    "product_version" => "2.0")

    # NetCDF CF standard name
    # http://cfconventions.org/Data/cf-standard-names/current/build/cf-standard-name-table.html
    # example "standard_name" = "sea_water_temperature",
    #"netcdf_standard_name" => "sea_water_salinity",

    #"netcdf_long_name" => "sea water salinity",

    #"netcdf_units" => "1e-3",

    # Abstract for the product
    #"abstract" => "...",

    # This option provides a place to acknowledge various types of support for the
    # project that produced the data
    #"acknowledgement" => "...",

    # Digital Object Identifier of the data product
    #"doi" => "...")

DataStructures.OrderedDict{String,String} with 7 entries:
  "project"         => "EMODnet Biology"
  "institution_urn" => "SDN::EDMO::422"
  "production"      => "Flanders Marine Institute"
  "Author_e-mail"   => "Lennert Schepers <lennert.schepers@vliz.be>"
  "source"          => "SAHFOS / Marine Biological Association (UK)"
  "comment"         => "..."
  "product_version" => "2.0"

In [10]:
@show yearlists[1]
@show monthlists[1]

yearlists[1] = 1958:1958
monthlists[1] = 

3-element Array{Int64,1}:
 1
 2
 3

[1, 2, 3]


In [ ]:
for varname = ["log_tem_lon","log_cal_hel", "log_tot_lar","log_tot_sma",
               "log_oit_spp", "log_cal_fin","ratio_large_to_small",
               "log_aca_spp", "log_met_luc", "log_chli"]

    varDir = joinpath(pwd(),varname)
    varncDir = joinpath(varDir,"netcdf_all")
    mkpath(varncDir)
    
    newe = readdlm(joinpath(varDir,"$(varname)_newe.txt"))
    newl = readdlm(joinpath(varDir,"$(varname)_newl.txt"))
    
    newl_avg=mean(filter(x -> x < 500_000, newl))
    newe_avg=mean(filter(x -> x < 15, newe))
    
    
    ncvarattrib = OrderedDict(
    "units" => "logab",
    "standard_name" => varname,
    "long_name" => "$(replace(varname,'_',' '))")
    
    @show varname
    t1 = now()
    @show t1
    
    # read data
    fname = "bigfile_$(varname).txt"

    if !isfile(fname)
        error("ERROR: File not found")
    else
        info("File found -> OK!")
    end

    value,lon,lat,depth,time,ids = loadbigfile(fname)
    checkobs((lon,lat,depth,time), value, ids)
    
    #### make selection for each timestep
    # yr = yearlists[1]
    # mo = monthlists[1]
    for yr = yearlists
        for mo = monthlists
                
            
                TS = TimeSelectorYearListMonthList([yr],[mo])
                @show TS
                t1 = now() # keep track of time
                @show t1

                # subset data based on timestep
                # https://stackoverflow.com/a/29661623
                sel = ones(length(value))
                sel = (depth .< 1) .& (indexin(Dates.month.(time), mo) .> 0) .& (indexin(Dates.year.(time), yr) .> 0)

                value_sel = value[sel]
                lon_sel = lon[sel]
                lat_sel = lat[sel]
                depth_sel = depth[sel]
                time_sel = time[sel]
                ids_sel = ids[sel];
                @show (size(value_sel))

               
                @show newl_avg
                @show newe_avg
            
                #### run diva
                if contains(varname, "log_")
                    shortname = split(varname, "log_")[2]
                else
                    shortname = varname
                end
            
                ncname = joinpath(varncDir,
                    "$(shortname)_$(yr[1])$(yr[length(yr)])_$(dec(mo[1],2))$(dec(mo[length(mo)],2)).nc");
                ncvarname = varname;
                @show ncname
            
                dbinfo = diva3d((lonr,latr,depthr, TS),        # TS: timestep -> NOW 1
                      (lon_sel,lat_sel,depth_sel,time_sel),    # observations
                      value_sel,                               # value of observations
                      (newl_avg,newl_avg,1.),                          # correlation length (in meters) # CAN NOT BE 0.
                      newe_avg,                                    # expected error 
                      ncname, ncvarname,                       # netcdf outputfilename, output variable name (in netcdf)
                      bathname = bathname,                     # bathymetry
                      bathisglobal = bathisglobal,             # global or not?
                      #plotres = plotres,                  # to plot or not to plot? [optional]
                      # plotres = plotres_timeindex,        # plot time index [optional]
                       ncvarattrib = ncvarattrib,           # [optional]
                       ncglobalattrib = ncglobalattrib,     # [optional]
                      memtofit = 1000             # [optional]
                )
            
                t2 = now()
                @show Dates.canonicalize(Dates.CompoundPeriod(t2-t1))

        end # mo
    end # yr
end # varname